In [ ]:
# This is the UI for pilot study

In [ ]:
%load_ext autoreload
%autoreload 2

# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torch.nn.functional as F
import torchvision
from torchvision import models
from torchvision import transforms
import matplotlib.image as mpimg
import timeit

labels_path = os.getenv("HOME") + '/.torch/models/imagenet_class_index.json'
with open(labels_path) as json_data:
    idx_to_labels = json.load(json_data)
    
################################ ABOVE FOR SHAP

import torch
import torchvision

torch.set_num_threads(1)

from torchvision.models import *
from visualisation.core.utils import device, image_net_postprocessing
from torch import nn
from operator import itemgetter
from visualisation.core.utils import imshow
from IPython.core.debugger import Tracer
from IPython.display import clear_output

task = 'Natural'
method = 'NNs' # Change this to Conf, EP or ... 

import csv
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch 
from utils import *
from PIL import Image

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

if method == 'NNs':
    size = 40
elif method == 'GradCAM' or method == 'EP' or method == 'SHAP' or method == 'PoolNet':
    size = 20
elif method == 'Conf':
    size = 10
else:
    raise ValueError('Not Implemented!')
    
answer_dict = dict()

file_name = 'Pilot_study2/{}/{}.csv'.format(task, method)
test_image_paths = glob.glob('Pilot_study2/{}/{}/*.*'.format(task, method))
test_images = list(map(lambda x: mpimg.imread(x), test_image_paths))
for idx, test_image in enumerate(test_images):
    print(idx)
    if idx == 10:
        break
    print(test_image_paths[idx])
    img_idx = test_image_paths[idx].split('.jpeg')[0].split('{}/'.format(method))[1]
    fig = plt.figure()
    plt.figure(figsize=(size,size))
    plt.axis('off')
    plt.imshow(test_image)
    plt.show()
    plt.close()

    start = timeit.default_timer()
    a = input("Press 1 if you agree, 0 if you disagree:")
    while a not in ['0', '1']:
        a = input("Input must be 0 or 1. Press 1 if you agree, 0 if you disagree:")
        
    stop = timeit.default_timer()
    time_taken = stop-start
    a = int(a)
    if a == 1:
        answer_dict[img_idx] = [1, 0, time_taken]
    else:
        answer_dict[img_idx] = [0, 1, time_taken]
    clear_output(wait=True)
    

with open(file_name, 'w') as file:
    for key in answer_dict.keys():
        file.write("%s,%s,%s,%s\n" %(key, answer_dict[key][0], answer_dict[key][1], answer_dict[key][2]))

# Evaluate accuracy
correct_images = glob.glob('/home/dexter/Downloads/Pilot_study2/{}/correct/*.*'.format(task))
wrong_images = glob.glob('/home/dexter/Downloads/Pilot_study2/{}/wrong/*.*'.format(task))

correct_indices = list()
wrong_indices = list()

gt_dict = dict()

for correct_image in correct_images:
    image_idx = (correct_image.split('.jpeg')[0]).split('val_')[1]
    correct_indices.append(image_idx)
    
gt_dict = dict(zip(correct_indices, [1]*len(correct_indices)))

for wrong_image in wrong_images:
    image_idx = (wrong_image.split('.jpeg')[0]).split('wrong/')[1]
    wrong_indices.append(image_idx)

gt_dict = {**dict(zip(correct_indices, [True]*len(correct_indices))), **dict(zip(wrong_indices, [False]*len(wrong_indices)))}

results = {}
reader = csv.reader(open(file_name))

time_taken = list()
for idx, row in enumerate(reader):
    key = row[0]
    time_taken.append(float(row[3]))
    if row[1] == '1':
        results[key] = True
    elif row[2] == '1':
        results[key] = False
    else:
        print(row)
        raise ValueError('Missing data in this row!')

result_temp = dict()
for key, val in results.items():
    k = key.split(".jpeg")[0]
    result_temp[k] = val
    

# Compute accuracy
shared_items = {k: gt_dict[k] for k in gt_dict if k in result_temp and gt_dict[k] == result_temp[k]}
print("The total number of correct answers is: {}".format(len(shared_items)))
print("The max time taken for an image is: {}".format(max(time_taken)))
print("The avg time taken for an image is: {}".format(sum(time_taken)/len(time_taken)))

In [ ]:
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
%load_ext autoreload
%autoreload 2


import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torch.nn.functional as F
import torchvision
from torchvision import models
from torchvision import transforms
import matplotlib.image as mpimg
import timeit

labels_path = os.getenv("HOME") + '/.torch/models/imagenet_class_index.json'
with open(labels_path) as json_data:
    idx_to_labels = json.load(json_data)
    
################################ ABOVE FOR SHAP

import torch
import torchvision

torch.set_num_threads(1)

from torchvision.models import *
from visualisation.core.utils import device, image_net_postprocessing
from torch import nn
from operator import itemgetter
from visualisation.core.utils import imshow
from IPython.core.debugger import Tracer
from IPython.display import clear_output

task = 'Dog'
img_per_method = 5

import csv
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch 
import random
from utils import *
from PIL import Image

adv_correct_desc = 0    
answer_dict = dict()
test_image_paths = list()

file_name = 'Pilot_study2/{}.csv'.format(task)
test_folders = glob.glob('Pilot_study2/{}/*'.format(task))
for test_folder in test_folders:
    test_image_paths += glob.glob(test_folder + '/*.*')
test_image_paths = random.sample(test_image_paths, len(test_image_paths)) # Shuffle the list

test_folders = glob.glob('Pilot_study2/Adversarial_{}/*'.format(task))
for test_folder in test_folders:
    test_image_paths += glob.glob(test_folder + '/*.*')
test_image_paths = random.sample(test_image_paths, len(test_image_paths)) # Shuffle the list
# test_image_paths = test_image_paths[:10]
test_images = list(map(lambda x: mpimg.imread(x), test_image_paths))
for idx, test_image in enumerate(test_images):
#     if idx == 10:
#         break
    print("{}/{}".format(idx+1, len(test_images)))
    print(test_image_paths[idx])
    task_method = test_image_paths[idx].split('Pilot_study2/')[1].split('val_')[0]
        
    if 'NNs' in task_method:
        size = 40
    elif 'GradCAM' in task_method or 'EP' in task_method or 'SHAP' in task_method or 'PoolNet' in task_method:
        size = 20
    elif 'Conf' in task_method:
        size = 10
    else:
        raise ValueError('Not Implemented!')
        
    img_idx = test_image_paths[idx].split('.jpeg')[0].split(task_method)[1]
    img_idx = task_method + img_idx

    fig = plt.figure()
    plt.figure(figsize=(size,size))
    plt.axis('off')
    plt.imshow(test_image)
    plt.show()
    plt.close()

    start = timeit.default_timer()
    a = input("Press 1 if you agree, 0 if you disagree:")
    while a not in ['0', '1']:
        a = input("Input must be 0 or 1. Press 1 if you agree, 0 if you disagree:")
        
    stop = timeit.default_timer()
    time_taken = stop-start
    a = int(a)
    if a == 1:
        answer_dict[img_idx] = [1, 0, time_taken]
    else:
        answer_dict[img_idx] = [0, 1, time_taken]
    clear_output(wait=True)
    
with open(file_name, 'w') as file:
    for key in answer_dict.keys():
        file.write("%s,%s,%s,%s\n" %(key, answer_dict[key][0], answer_dict[key][1], answer_dict[key][2]))

In [ ]:
task = 'Dog'
file_name = 'Pilot_study2/{}.csv'.format(task)

reader = csv.reader(open(file_name))
answer_dict = dict()
time_taken = list()
for idx, row in enumerate(reader):
    answer_dict[row[0]] = row[1:]
    time_taken.append(float(row[3]))

# Evaluate accuracy
correct_images = glob.glob('/home/dexter/Downloads/Pilot_study2/{}/correct_images/*.*'.format(task))
wrong_images = glob.glob('/home/dexter/Downloads/Pilot_study2/{}/wrong_images/*.*'.format(task))

corrects_bin1 = dict(zip([x.split('.jpeg')[0].split('correct_bin1_images/')[1] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/{}/correct_bin1_images/*.*'.format(task))], [True]*img_per_method*7))
corrects_bin2 = dict(zip([x.split('.jpeg')[0].split('correct_bin2_images/')[1] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/{}/correct_bin2_images/*.*'.format(task))], [True]*img_per_method*7))
corrects_bin3 = dict(zip([x.split('.jpeg')[0].split('correct_bin3_images/')[1] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/{}/correct_bin3_images/*.*'.format(task))], [True]*img_per_method*7))

wrongs_bin1 = dict(zip([x.split('.jpeg')[0].split('wrong_bin1_images/')[1] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/{}/wrong_bin1_images/*.*'.format(task))], [False]*img_per_method*7))
wrongs_bin2 = dict(zip([x.split('.jpeg')[0].split('wrong_bin2_images/')[1] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/{}/wrong_bin2_images/*.*'.format(task))], [False]*img_per_method*7))
wrongs_bin3 = dict(zip([x.split('.jpeg')[0].split('wrong_bin3_images/')[1] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/{}/wrong_bin3_images/*.*'.format(task))], [False]*img_per_method*7))

adv_wrongs_bin1 = dict(zip([x.split('.jpeg')[0].split('ILSVRC2012_')[1][:22] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/Adversarial_{}/wrong_bin1_images/*.*'.format('Dog'))], [False]*img_per_method*7))
adv_wrongs_bin2 = dict(zip([x.split('.jpeg')[0].split('ILSVRC2012_')[1][:22] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/Adversarial_{}/wrong_bin2_images/*.*'.format('Dog'))], [False]*img_per_method*7))
adv_wrongs_bin3 = dict(zip([x.split('.jpeg')[0].split('ILSVRC2012_')[1][:22] for x in glob.glob('/home/dexter/Downloads/Pilot_study2/Adversarial_{}/wrong_bin3_images/*.*'.format('Dog'))], [False]*img_per_method*7))

correct_indices = list()
wrong_indices = list()

gt_dict = dict()

for correct_image in correct_images:
    image_idx = (correct_image.split('.jpeg')[0]).split('correct_images/')[1]
    correct_indices.append(image_idx)
    
gt_dict = dict(zip(correct_indices, [1]*len(correct_indices)))

for wrong_image in wrong_images:
    image_idx = (wrong_image.split('.jpeg')[0]).split('wrong_images/')[1]
    wrong_indices.append(image_idx)

gt_dict = {**dict(zip(correct_indices, [True]*len(correct_indices))), **dict(zip(wrong_indices, [False]*len(wrong_indices)))}

print("The max time taken for an image is: {}".format(max(time_taken)))
print("The avg time taken for an image is: {}".format(sum(time_taken)/len(time_taken)))

In [ ]:
# Tracer()()
methods = ['Conf', 'EP', 'GradCAM', 'NNs', 'PoolNet', 'SHAP'] # alphabettically sorted
# print(sorted(answer_dict.keys()))
adv_keys = sorted(answer_dict.keys())[:img_per_method*3*6] # 3 bins 6 methods
unattacked_keys = sorted(answer_dict.keys())[img_per_method*3*6:img_per_method*3*6 + img_per_method*2*3*6] # correct/wrong 3 bins 6 methods
adv_dict = dict()
unattacked_dict = dict()
for method in methods:
    tmp_dict = dict()
    time_taken = list()
    for k in adv_keys[methods.index(method)*img_per_method*3:methods.index(method)*img_per_method*3 + img_per_method*3]:
        key = k.split('ILSVRC2012_')[1][:22]
        time_taken.append(float(answer_dict[k][2]))
        if answer_dict[k][0] == '1':
            value = True
        elif answer_dict[k][1] == '1':
            value = False
        tmp_dict[key] = value
    adv_dict[method] = tmp_dict
    method_result_adv = adv_dict[method]
    
    tmp_dict = dict()
    time_taken = list()
    for k in unattacked_keys[methods.index(method)*img_per_method*2*3:methods.index(method)*img_per_method*2*3 + img_per_method*2*3]:
        key = k.split(method+'/')[1]
        time_taken.append(float(answer_dict[k][2]))
        if answer_dict[k][0] == '1':
            value = True
        elif answer_dict[k][1] == '1':
            value = False
        tmp_dict[key] = value
    unattacked_dict[method] = tmp_dict
    method_result_unattacked = unattacked_dict[method]
    shared_items = {k: gt_dict[k] for k in gt_dict if k in method_result_unattacked and gt_dict[k] == method_result_unattacked[k]}

print("{}".format(task))
print("Results on unattacked images:")
for method in methods:
    method_result_unattacked = unattacked_dict[method]
    shared_items = {k: gt_dict[k] for k in gt_dict if k in method_result_unattacked and gt_dict[k] == method_result_unattacked[k]}
    print('{}: {}/{}'.format(method,len(shared_items),img_per_method*2*3))
    
print("Results on adversarial images:")
for method in methods:
    method_result_adv = adv_dict[method]
    print('{}: {}/{}'.format(method, sum(value == 0 for value in method_result_adv.values()), img_per_method*3))